In [98]:
import os
import time
import requests
import warnings
import langchain
from langchain.llms import OpenAI
from langchain.agents import  Tool, initialize_agent

# os.environ["OPENAI_API_KEY"] = "sk-proj-OO0gcL1b27GBMkKp9gMtT3BlbkFJyg5bUhILYYgmJsn0sm0d"
os.environ["OPENAI_API_KEY"] = "sk-proj-KsduOxFQLOiaOxxoBCKiT3BlbkFJ2bgEqAMPbqmm3zy2n4Hq"
os.environ["COINMARKETCAP_API_KEY"] = "1491e542-5af4-49b6-93bb-77df1a196765"

warnings.filterwarnings("ignore")

llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-16k-0613")


In [103]:
import os
import requests
import random
import json

os.environ["COINMARKETCAP_API_KEY"] = "1491e542-5af4-49b6-93bb-77df1a196765"
# Define the list of questions
questions = [
    "What is today's price of {cryptocurrency_name}?",
    "How much does {cryptocurrency_name} cost right now?",
    "What is the current market trend for {cryptocurrency_name}?",
    "Is {cryptocurrency_name} going up or down today?",
    "What was the price of {cryptocurrency_name} one month ago?",
    "How has {cryptocurrency_name} performed over the past year?",
    "What is the market cap of {cryptocurrency_name}?",
    "How does {cryptocurrency_name}'s market cap compare to other cryptocurrencies?",
    "What is the 24-hour trading volume for {cryptocurrency_name}?",
    "How liquid is {cryptocurrency_name} in the current market?",
    "Is {cryptocurrency_name} a good investment right now?",
    "Should I buy or sell {cryptocurrency_name}?",
    "What are the predictions for {cryptocurrency_name}'s price in the next six months?",
    "Will {cryptocurrency_name} reach a new all-time high soon?",
    "What are the latest technical analysis indicators for {cryptocurrency_name}?",
    "Can you provide a technical analysis of {cryptocurrency_name}?",
    "How does {cryptocurrency_name} compare to Bitcoin in terms of performance?",
    "Which is better to invest in, {cryptocurrency_name} or Ethereum?"
]

# Function to fetch top 1000 cryptocurrencies
def get_top_cryptocurrencies(limit=1000):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
    params = {
        "start": "1",
        "limit": str(limit),
        "convert": "USD"
    }
    headers = {
        "X-CMC_PRO_API_KEY": os.getenv("COINMARKETCAP_API_KEY")
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if 'data' in data:
        return [crypto['name'] for crypto in data['data']]
    else:
        raise Exception("Failed to fetch data from CoinMarketCap API")

# Generate dataset of questions
def generate_question_dataset(cryptocurrency_names, questions, num_samples=1000):
    dataset = []
    for _ in range(num_samples):
        crypto_name = random.choice(cryptocurrency_names)
        question_template = random.choice(questions)
        question = question_template.format(cryptocurrency_name=crypto_name)
        dataset.append(question)
    return dataset

# Fetch top 1000 cryptocurrencies
try:
    top_cryptocurrencies = get_top_cryptocurrencies()
    print(f"Fetched {len(top_cryptocurrencies)} cryptocurrencies.")
except Exception as e:
    print(f"Error fetching cryptocurrencies: {e}")
    top_cryptocurrencies = []

# Generate and save dataset
if top_cryptocurrencies:
    question_dataset = generate_question_dataset(top_cryptocurrencies, questions, num_samples=1000)
    with open("cryptocurrency_questions_dataset.json", "w") as f:
        json.dump(question_dataset, f, indent=2)
    print("Dataset generated and saved to 'cryptocurrency_questions_dataset.json'")
else:
    print("Failed to generate dataset due to empty cryptocurrency list.")


Fetched 1000 cryptocurrencies.
Dataset generated and saved to 'cryptocurrency_questions_dataset.json'


In [105]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random
import requests
import os
import json

# Load the dataset
with open("cryptocurrency_questions_dataset.json", "r") as f:
    question_dataset = json.load(f)

# Mock function to generate annotations for the training data
def create_train_data(questions, crypto_names):
    train_data = []
    for question in questions:
        entities = []
        for name in crypto_names:
            start = question.find(name)
            if start != -1:
                end = start + len(name)
                entities.append((start, end, "CRYPTO"))
        entities = filter_overlapping_entities(entities)
        train_data.append((question, {"entities": entities}))
    return train_data

def filter_overlapping_entities(entities):
    # Sort entities by start position
    entities = sorted(entities, key=lambda x: x[0])
    filtered_entities = []
    current_end = -1
    for start, end, label in entities:
        if start >= current_end:
            filtered_entities.append((start, end, label))
            current_end = end
    return filtered_entities

# Fetch top 1000 cryptocurrencies again
def get_top_cryptocurrencies(limit=1000):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
    params = {
        "start": "1",
        "limit": str(limit),
        "convert": "USD"
    }
    headers = {
        "X-CMC_PRO_API_KEY": os.getenv("COINMARKETCAP_API_KEY")
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if 'data' in data:
        return [crypto['name'] for crypto in data['data']]
    else:
        raise Exception("Failed to fetch data from CoinMarketCap API")

try:
    top_cryptocurrencies = get_top_cryptocurrencies()
    print(f"Fetched {len(top_cryptocurrencies)} cryptocurrencies.")
except Exception as e:
    print(f"Error fetching cryptocurrencies: {e}")
    top_cryptocurrencies = []

# Create training data
TRAIN_DATA = create_train_data(question_dataset, top_cryptocurrencies)

# Initialize the blank spacy model
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Train the NER model
optimizer = nlp.begin_training()
for i in range(20):
    random.shuffle(TRAIN_DATA)
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, sgd=optimizer)




Fetched 1000 cryptocurrencies.


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path\\to\\custom_model'

In [107]:
# Save the model
nlp.to_disk(".")



In [163]:
# Load and use the custom model
nlp = spacy.load(".")

# Test the model
test_text = "Current market trend of ?"
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Entities in 'Current market trend of BTC?'
BTC 24 27 CRYPTO


In [53]:
# def get_crypto_price(crypto_symbol, history=5):
#     url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
#     params = {
#         "symbol": crypto_symbol,
#         "count": history,
#         "interval": "daily",
#         "convert": "USD"
#     }
#     headers = {
#         "X-CMC_PRO_API_KEY": os.getenv("COINMARKETCAP_API_KEY")
#     }
    
#     response = requests.get(url, headers=headers, params=params)
#     data = response.json()
    
#     if 'data' in data:
#         quotes = data['data']['quotes']
#         result = []
#         for quote in quotes:
#             date = quote['timestamp'].split('T')[0]
#             price = quote['quote']['USD']['price']
#             volume = quote['quote']['USD']['volume_24h']
#             result.append(f"Date: {date}, Close: {price:.2f}, Volume: {volume:.2f}")
#         return "\n".join(result)
#     else:
#         return "No data found"

# print(get_crypto_price("BTC"))


No data found


In [139]:
def get_crypto_details(crypto_symbol):
    url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
    params = {
        "symbol": crypto_symbol
    }
    headers = {
        "X-CMC_PRO_API_KEY": os.getenv("COINMARKETCAP_API_KEY")
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    crypto_symbol = crypto_symbol.upper()
    
    if 'data' in data:
        crypto_data = list(data['data'].values())[0]  # Extract the first (and presumably only) item
        details = {
            "Name": crypto_data['name'],
            "Symbol": crypto_data['symbol'],
            "Category": crypto_data['category'],
            "Description": crypto_data['description']
        }
        details_str = "\n".join([f"{key}: {value}" for key, value in details.items()])
        return details_str
    else:
        return "No data found"

print(get_crypto_details("BTC"))


No data found


In [158]:
def get_latest_crypto_data(crypto_symbol):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
    params = {
        "start": "1",
        "limit": "100",
        "convert": "USD",
    }
    headers = {
        "X-CMC_PRO_API_KEY": os.getenv("COINMARKETCAP_API_KEY")
    }
    crypto_symbol = crypto_symbol.upper()
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if 'data' in data:
        for crypto in data['data']:
            if crypto['symbol'] == crypto_symbol:
                market_data = {
                    "Price (USD)": crypto['quote']['USD']['price'],
                    "Volume 24h (USD)": crypto['quote']['USD']['volume_24h'],
                    "Market Cap (USD)": crypto['quote']['USD']['market_cap'],
                    "Max Supply": crypto['max_supply'],
                    "Circulating Supply": crypto['circulating_supply'],
                    "Total Supply": crypto['total_supply']
                }
                return market_data['Price (USD)']
        return "Cryptocurrency not found"
    else:
        return "No data found"

print(get_latest_crypto_data("BTC"))


66661.36273758255


In [157]:
import requests
import os

def get_crypto_symbol(slug):
    url = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/info"
    params = {
        "slug": slug
    }
    headers = {
        "X-CMC_PRO_API_KEY": os.getenv("COINMARKETCAP_API_KEY")
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if 'data' in data:
            crypto_data = data['data']
            for key, value in crypto_data.items():
                if value['slug'] == slug:
                    return value['symbol']
            return "Cryptocurrency not found"
        else:
            return "No data found"
    except Exception as e:
        return f"Error occurred: {e}"

# Example usage:
symbol = get_crypto_symbol("bitcoin")




The symbol for Bitcoin is BTC.
The symbol for Ethereum is ETH.
DOGE


In [160]:
def get_latest_crypto_data(crypto_symbol):
    crypto_symbol = crypto_symbol.upper()
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
    params = {
        "start": "1",
        "limit": "100",
        "convert": "USD"
    }
    headers = {
        "X-CMC_PRO_API_KEY": os.getenv("COINMARKETCAP_API_KEY")
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if 'data' in data:
        for crypto in data['data']:
            if crypto['symbol'] == crypto_symbol:
                market_data = {
                    "Name": crypto['name'],
                    "Symbol": crypto['symbol'],
                    "Price (USD)": crypto['quote']['USD']['price'],
                    "Volume 24h (USD)": crypto['quote']['USD']['volume_24h'],
                    "Market Cap (USD)": crypto['quote']['USD']['market_cap'],
                    "Max Supply": crypto['max_supply'],
                    "Circulating Supply": crypto['circulating_supply'],
                    "Total Supply": crypto['total_supply'],
                }
                return market_data
        return "Cryptocurrency not found"
    else:
        return "No data found"
    
get_latest_crypto_data("btc")

{'Name': 'Bitcoin',
 'Symbol': 'BTC',
 'Price (USD)': 66663.0338195427,
 'Volume 24h (USD)': 20546123652.90494,
 'Market Cap (USD)': 1313311296879.1191,
 'Max Supply': 21000000,
 'Circulating Supply': 19700743,
 'Total Supply': 19700743}

In [101]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="get latest crypto data",
        func=get_latest_crypto_data,
        description="Use this when you are asked to get details on current price and other metrics about a cryptocurrency. You should input the cryptocurrency symbol to it."
    ),
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Use this to fetch recent news about cryptocurrencies. You can input the cryptocurrency name to it."
    ),
]

zero_shot_agent = initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)


# Define the new prompt with more specific instructions
new_prompt = """
You are a financial advisor. Give cryptocurrency recommendations for given query.
Every time first you should identify the cryptocurrency name and get its symbol. Remember symbol is always in uppercase.
Answer the following questions as best you can. You have access to the following tools:

get latest crypto data: Use this when you are asked to get details on current price and other metrics about a cryptocurrency. You should input the cryptocurrency symbol to it.

DuckDuckGo Search: Use this to fetch recent news about cryptocurrencies. You can input the cryptocurrency name to it.

Steps:
1) Get the cryptocurrency name and symbol. Output - cryptocurrency symbol
2) Use "get latest crypto data" tool to gather cryptocurrency info. Output - Crypto data
3) Use "DuckDuckGo Search" tool to search for latest cryptocurrency-related news. Output - Crypto news
4) Analyze the cryptocurrency based on gathered data and give detailed analysis for investment choice only when asked by user. Provide numbers and reasons to justify your answer. Output - Detailed Crypto Analysis

Make sure to include an action after each observation.
Stop the iteration once a final answer is found.

Use the following format:

Question: the input question you must answer unless if it's not related to crypto you can stop there
Thought: you should always think about what to do, also try to follow steps mentioned above
Action: the action to take, should be one of [get latest crypto data, DuckDuckGo Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat maximum 5 times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""


In [84]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name = "get latest crypto data",
        func=  get_latest_crypto_data,
        description= "Use this when you are asked to get details on current price and other metrics about a cryptocurrency. You should input the cryptocurrency symbol to it."
    ),

    # Tool(
    #     name="get crypto data",
    #     func=get_crypto_price,
    #     description="Use when you are asked to evaluate or analyze a cryptocurrency. This will output historic price data. You should input the cryptocurrency symbol to it."
    # ),
    Tool(
        name="get crypto details",
        func=get_crypto_details,
        description="Use this to get details and metrics about a cryptocurrency. You should input the cryptocurrency symbol to it."
    ),
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Use this to fetch recent news about cryptocurrencies. You can input the cryptocurrency name to it."
    )
]

zero_shot_agent = initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

print(zero_shot_agent.agent.llm_chain.prompt.template)


Answer the following questions as best you can. You have access to the following tools:

get latest crypto data: Use this when you are asked to get details on current price and other metrics about a cryptocurrency. You should input the cryptocurrency symbol to it.
get crypto details: Use this to get details and metrics about a cryptocurrency. You should input the cryptocurrency symbol to it.
DuckDuckGo Search: Use this to fetch recent news about cryptocurrencies. You can input the cryptocurrency name to it.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [get latest crypto data, get crypto details, DuckDuckGo Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Q

In [85]:
new_prompt = """You are a financial advisor. Give cryptocurrency recommendations for given query.
Every time first you should identify the cryptocurrency name and get its symbol.Remember symbol is always in uppercase.
Answer the following questions as best you can. You have access to the following tools:

get latest crypto data: Use this when you are asked to get details on current price and other metrics about a cryptocurrency. You should input the cryptocurrency symbol to it.
get crypto data: Use when you are asked to evaluate or analyze a cryptocurrency. This will output historic price data. You should input the cryptocurrency symbol to it.
get crypto details: Use this to get details and metrics about a cryptocurrency. You should input the cryptocurrency symbol to it.
DuckDuckGo Search: Use this to fetch recent news about cryptocurrencies. You can input the cryptocurrency name to it.

steps-
Note- if you fail in satisfying any of the step below, just move to the next one and if you found answer just stop and give output. Only go to step 5 when Question demands for analysis.
1) Get the cryptocurrency name and symbol. Output - cryptocurrency symbol
2) Use "get latest crypto data" tool to gather cryptocurrency info. Output - Crypto data
3) Get cryptocurrency details using "get crypto details". Output - Crypto details
4) Use "DuckDuckGo Search" tool to search for latest cryptocurrency related news. Output - Crypto news
5) Analyze the cryptocurrency based on gathered data and give detailed analysis for investment choice only when asked by user. Provide numbers and reasons to justify your answer. Output - Detailed Crypto Analysis

Use the following format:

Question: the input question you must answer unless if it's not related to crypto you can stop there
Thought: you should always think about what to do, also try to follow steps mentioned above
Action: the action to take, should be one of [get latest crypto data, get crypto details, DuckDuckGo Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat maximum 5 times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!

Question: {input}
Thought:{agent_scratchpad}"""
#, get crypto data
zero_shot_agent.agent.llm_chain.prompt.template = new_prompt


In [ ]:
# def perform_llm_request(proxy: Proxy, user_input: str, uid: str) -> Union[str, None]:
#     proxy_response: str = ""
#     result_key = "outputText"
#     llm_request = make_llm_request(user_input)
#     try:
#         llm_output = proxy.request(vars(llm_request), uid)
#         llm_output.wait(timeout=10)

#         if llm_output.status() == "COMPLETED":
#             proxy_response = llm_output.data()[result_key]
#             if "Final Answer:" in proxy_response:
#                 return proxy_response.split("Final Answer:")[1].strip()

#         else:
#             proxy_response = "Error: LLM did not complete successfully."

#     except Exception as e:
#         proxy_response = f"Error: {str(e)}"

#     return proxy_response


In [102]:
question = "What do you think about dogecoin is it bullish or bearish ?"
result = zero_shot_agent(question)




> Entering new AgentExecutor chain...


Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retr

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [61]:
print(result["output"])

The price of Dogecoin today is $0.14998762423818088.


In [ ]:
# I have gathered all the necessary information about Dogecoin. Now I can determine if it is bullish or bearish.
# Final Answer: Based on the gathered data, Dogecoin (DOGE) is currently priced at $0.1548919166037079 and has experienced a 3.69% increase in the last 24 hours. It is trading on 965 active markets with a total trading volume of $1,852,765,378.20. Additionally, recent news suggests that Dogecoin has reached its highest level since December 2021 and there are rumors about its implementation on a social media platform owned by Elon Musk. However, it is important to note that the price action of DOGE is overbought and may reverse. Therefore, it is recommended to exercise caution and further analyze the market trends before making any investment decisions.
